# SQL Joins Demo - lecture 5


Jupyter/IPython notebooks can be used for an interactive data analysis with SQL on a relational database. This fuses together the advantages of using Jupyter, a well-established platform for data analysis, with the ease of use of SQL and the performance of SQL engines.




Useful links:
https://www.geeksforgeeks.org/how-to-install-ipython-sql-package-in-jupyter-notebook/




In [ ]:
import pandas as pd

# Create two dataframes to be joined
df1 = pd.DataFrame({'key': ['A', 'B', 'C', 'D'],
                    'value': [1, 2, 3, 4]})

df2 = pd.DataFrame({'key': ['B', 'D', 'E', 'F'],
                    'value': [5, 6, 7, 8]})

# Perform a left join using the key column
result = pd.merge(df1, df2, on='key', how='left', indicator=True)

print(result)


  key  value_x  value_y     _merge
0   A        1      NaN  left_only
1   B        2      5.0       both
2   C        3      NaN  left_only
3   D        4      6.0       both


If you are not using Colab, you will have to install ipython-sql using the following command:

In [ ]:
#pip install ipython-sql

Use %load_ext sql. to load the SQL module in the notebook. Now, with the use of %sql magic, you can use SQL queries directly in Jupyter Notebook. Using ipython-sql in Jupyter Notebook

In [ ]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Once you have loaded the sql extension, you can interact with it after initializing connection to SQLite. If you are on your own Jupyter installation not on Colab, you will have to install SQLite and it’s driver. On Colab, you can simply connect to an SQLite database using the following command.

In [ ]:
%%sql
sqlite:///dsc100.db
PRAGMA foreign_keys=ON;


Done.


[]

In [ ]:
%%sql
CREATE TABLE Employees (
ID INT PRIMARY KEY,
Name VARCHAR(255),
ManagerID INT,
FOREIGN KEY (ManagerID) REFERENCES Employees(ID)
)

   sqlite:///dsc100.db
 * sqlite:///test.db
Done.


[]

This is an exmpale of a self-reference. A self-reference is a reference within a table to another record within the same table. In this case, the Employees table has a Manager ID column which refers to the ID of another employee within the same table. This allows for the creation of a hierarchy within the table, where each employee has a manager and potentially multiple subordinates. This type of relationship is commonly used in organizational charts and other hierarchical structures.


Now let's add some tuples in this dataset:

In [ ]:
%%sql
INSERT INTO Employees (ID, Name, ManagerID)
VALUES (1, 'John Smith', 3);


   sqlite:///dsc100.db
 * sqlite:///test.db


IntegrityError: ignored

**But this leads to violation of foregn key constraint. How on earth can we insert data into such a self refrenceing databses? 😖😖😖**


In this example, you can set the ManagerID column  to null. Once you have inserted some data into the table, you can then use the ManagerID column to reference the ID of the employee's manager.

In [ ]:
%%sql
INSERT INTO Employees (ID, Name, ManagerID) VALUES (1, 'John Doe', null);


   sqlite:///dsc100.db
 * sqlite:///test.db
1 rows affected.


[]

In [ ]:
%%sql
SELECT * FROM Employees;

   sqlite:///dsc100.db
 * sqlite:///test.db
Done.


ID,Name,ManagerID
1,John Doe,None


In [ ]:
%%sql
DELETE FROM Employees;

   sqlite:///dsc100.db
 * sqlite:///test.db
1 rows affected.


[]

In [ ]:
%%sql

INSERT INTO Employees (ID, Name, ManagerID)
VALUES (1, 'John Smith', 3),
(2, 'Jane Doe', 3),
(3, 'Bob Johnson',  ),
(4, 'Emily Davis', 1),
(5, 'Michael Brown', 2);


   sqlite:///dsc100.db
 * sqlite:///test.db
5 rows affected.


[]

In [ ]:
%%sql
SELECT * FROM Employees;

   sqlite:///dsc100.db
 * sqlite:///test.db
Done.


ID,Name,ManagerID
1,John Smith,3
2,Jane Doe,3
3,Bob Johnson,None
4,Emily Davis,1
5,Michael Brown,2


In [ ]:
%%sql
SELECT * FROM Employees;

 * sqlite:///dsc100.db
Done.


ID,Name,ManagerID
1,John Smith,3
2,Jane Doe,3
3,Bob Johnson,None
4,Emily Davis,1
5,Michael Brown,2


In [ ]:
%%sql
SELECT *
FROM Employees as e1
JOIN Employees as e2
ON e1.ManagerID = e2.ID;



   sqlite:///dsc100.db
 * sqlite:///test.db
Done.


ID,Name,ManagerID,ID_1,Name_1,ManagerID_1
1,John Smith,3,3,Bob Johnson,None
2,Jane Doe,3,3,Bob Johnson,None
4,Emily Davis,1,1,John Smith,3
5,Michael Brown,2,2,Jane Doe,3


Retrieve all employees and their corresponding managers:

In [ ]:
%%sql
SELECT e1.Name as Employee, e2.Name as Manager
FROM Employees as e1
JOIN Employees as e2
ON e1.ManagerID = e2.ID


   sqlite:///dsc100.db
 * sqlite:///test.db
Done.


Employee,Manager
John Smith,Bob Johnson
Jane Doe,Bob Johnson
Emily Davis,John Smith
Michael Brown,Jane Doe


In [ ]:
%%sql SELECT *
FROM Employees;

   sqlite:///dsc100.db
 * sqlite:///test.db
Done.


ID,Name,ManagerID
1,John Smith,3
2,Jane Doe,3
3,Bob Johnson,None
4,Emily Davis,1
5,Michael Brown,2


Retrieve all employees and their corresponding managers, but also include those employees who do not have a manager:

In [ ]:
%%sql
SELECT * from Employee;

   sqlite:///dsc100.db
 * sqlite:///test.db
(sqlite3.OperationalError) no such table: Employee
[SQL: SELECT * from Employee;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [ ]:
%%sql
SELECT e1.Name as Employee, e2.Name as Manager
FROM  Employees e1
LEFT OUTER JOIN Employees e2
ON e1.ManagerID = e2.ID


   sqlite:///dsc100.db
 * sqlite:///test.db
Done.


Employee,Manager
John Smith,Bob Johnson
Jane Doe,Bob Johnson
Bob Johnson,None
Emily Davis,John Smith
Michael Brown,Jane Doe


Retrieve all employees and their corresponding managers, but also include those mannagers who do not have any employees yet:

In [ ]:
%%sql
SELECT e1.Name as Employee, e2.Name as Manager
FROM Employees e1
RIGHT JOIN Employees e2
ON e1.ManagerID = e2.ID


   sqlite:///dsc100.db
 * sqlite:///test.db
(sqlite3.OperationalError) RIGHT and FULL OUTER JOINs are not currently supported
[SQL: SELECT e1.Name as Employee, e2.Name as Manager
FROM Employees e1
RIGHT JOIN Employees e2
ON e1.ManagerID = e2.ID]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


But how can we do this now?! 😞

In [ ]:
%%sql
SELECT e1.Name as Employee, e2.Name as Manager
FROM Employees e2
LEFT JOIN Employees e1
ON e1.ManagerID = e2.ID


   sqlite:///dsc100.db
 * sqlite:///test.db
Done.


Employee,Manager
Emily Davis,John Smith
Michael Brown,Jane Doe
Jane Doe,Bob Johnson
John Smith,Bob Johnson
None,Emily Davis
None,Michael Brown


Retrieve all employees and their corresponding managers, but also include those employees and managers who do not have a match in the other table:


In [ ]:
%%sql SELECT e1.Name as Employee, e2.Name as Manager
FROM Employees e1
FULL OUTER JOIN Employees e2
ON e1.ManagerID = e2.ID


   sqlite:///dsc100.db
 * sqlite:///test.db
(sqlite3.OperationalError) RIGHT and FULL OUTER JOINs are not currently supported
[SQL: SELECT e1.Name as Employee, e2.Name as Manager
FROM Employees e1
FULL OUTER JOIN Employees e2
ON e1.ManagerID = e2.ID]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


Can we capture full outer join using left thou?  😥 😥 😥

In [ ]:
%%sql
SELECT e1.Name as Employee, e2.Name as Manager
FROM Employees e2
LEft JOIN Employees e1
ON e1.ManagerID = e2.ID
UNION
SELECT e1.Name as Employee, e2.Name as Manager
FROM Employees e1
LEft JOIN Employees e2
ON e1.ManagerID = e2.ID

   sqlite:///dsc100.db
 * sqlite:///test.db
Done.


Employee,Manager
None,Emily Davis
None,Michael Brown
Bob Johnson,None
Emily Davis,John Smith
Jane Doe,Bob Johnson
John Smith,Bob Johnson
Michael Brown,Jane Doe


In [ ]:
%%sql
SELECT * FROM Employees;

   sqlite:///dsc100.db
 * sqlite:///test.db
Done.


ID,Name,ManagerID
1,John Smith,3
2,Jane Doe,3
3,Bob Johnson,None
4,Emily Davis,1
5,Michael Brown,2


Retrieve all employees who do not have a manager:

In [ ]:
%%sql
SELECT Name
FROM Employees
WHERE ManagerID IS NULL



 * sqlite:///dsc100.db
Done.


Name
Bob Johnson


Retrieve the names of all employees who are also managers

In [ ]:
%%sql
SELECT DISTINCT E1.Name as name
FROM Employees E1
JOIN Employees E2
ON E1.ID = E2.ManagerID

 * sqlite:///dsc100.db
Done.


name
Bob Johnson
John Smith
Jane Doe


In [ ]:
%%sql
SELECT Name FROM Employees WHERE ID IN (SELECT ManagerID FROM Employees);

   sqlite:///dsc100.db
 * sqlite:///test.db
Done.


Name
John Smith
Jane Doe
Bob Johnson


In [ ]:
%%sql

-- create the Movie table
CREATE TABLE Movie (
    name varchar(50) PRIMARY KEY,
    year integer,
    genre varchar(50)
);

-- insert data into the Movie table
INSERT INTO Movie (name, year, genre)
VALUES ('Apocalypse Now', 1979, 'War'),
       ('The Godfather', 1972, 'Crime'),
       ('Planet Earth II', 2016, 'Nature Documentary'),
       ('House of Gucci', 2021, 'Crime');

-- create the ActedIn table
CREATE TABLE ActedIn (
    actorname varchar(20),
    moviename varchar(20) REFERENCES Movie,
    PRIMARY KEY (actorname, moviename)
);

-- insert data into the ActedIn table
INSERT INTO ActedIn (actorname, moviename)
VALUES ('Marlon Brando', 'Apocalypse Now'),
       ('Al Pacino', 'The Godfather'),
       ('Marlon Brando', 'The Godfather'),
       ('Robert De Niro', 'House of Gucci');


   sqlite:///dsc100.db
 * sqlite:///test.db
Done.
Done.
Done.
Done.


[]

In [ ]:
%%sql
SELECT * FROM Movie;

   sqlite:///dsc100.db
 * sqlite:///test.db
Done.


name,year,genre
Apocalypse Now,1979,War
The Godfather,1972,Crime
Planet Earth II,2016,Nature Documentary
House of Gucci,2021,Crime


In [ ]:
%%sql
SELECT * FROM ActedIn;

   sqlite:///dsc100.db
 * sqlite:///test.db
Done.


actorname,moviename
Marlon Brando,Apocalypse Now
Al Pacino,The Godfather
Marlon Brando,The Godfather
Robert De Niro,House of Gucci


Retrieve the names of movies and the corresponding actors who acted in them:

In [ ]:
%%sql
SELECT Movie.name, ActedIn.actorname
FROM Movie
JOIN ActedIn ON Movie.name = ActedIn.moviename;

   sqlite:///dsc100.db
 * sqlite:///test.db
Done.


name,actorname
Apocalypse Now,Marlon Brando
The Godfather,Al Pacino
The Godfather,Marlon Brando
House of Gucci,Robert De Niro


In [ ]:
%%sql
SELECT Movie.name, ActedIn.actorname
FROM Movie JOIN ActedIn;

   sqlite:///dsc100.db
 * sqlite:///test.db
Done.


name,year,genre,actorname,moviename
Apocalypse Now,1979,War,Marlon Brando,Apocalypse Now
Apocalypse Now,1979,War,Al Pacino,The Godfather
Apocalypse Now,1979,War,Marlon Brando,The Godfather
Apocalypse Now,1979,War,Robert De Niro,House of Gucci
The Godfather,1972,Crime,Marlon Brando,Apocalypse Now
The Godfather,1972,Crime,Al Pacino,The Godfather
The Godfather,1972,Crime,Marlon Brando,The Godfather
The Godfather,1972,Crime,Robert De Niro,House of Gucci
Planet Earth II,2016,Nature Documentary,Marlon Brando,Apocalypse Now
Planet Earth II,2016,Nature Documentary,Al Pacino,The Godfather


In [ ]:
%%sql
SELECT *
FROM Movie , ActedIn;

 * sqlite:///dsc100.db
Done.


name,year,genre,actorname,moviename
Apocalypse Now,1979,War,Marlon Brando,Apocalypse Now
Apocalypse Now,1979,War,Al Pacino,The Godfather
Apocalypse Now,1979,War,Marlon Brando,The Godfather
Apocalypse Now,1979,War,Robert De Niro,House of Gucci
The Godfather,1972,Crime,Marlon Brando,Apocalypse Now
The Godfather,1972,Crime,Al Pacino,The Godfather
The Godfather,1972,Crime,Marlon Brando,The Godfather
The Godfather,1972,Crime,Robert De Niro,House of Gucci
Planet Earth II,2016,Nature Documentary,Marlon Brando,Apocalypse Now
Planet Earth II,2016,Nature Documentary,Al Pacino,The Godfather


In [ ]:
%%sql
SELECT *
FROM ActedIn a1
JOIN ActedIn a2
ON a1.actorname = a2.actorname;
WHERE a1.moviename = 'Apocalypse now' AND a2.moviename = 'The god father';


   sqlite:///dsc100.db
 * sqlite:///test.db
Done.
(sqlite3.OperationalError) near "WHERE": syntax error
[SQL: WHERE a1.moviename = 'Apocalypse now' AND a2.moviename = 'The god father';]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


Retrieve the names of actors who acted in both 'Apocalypse now' and 'The god father':

In [ ]:
%%sql
SELECT DISTINCT a1.actorname
FROM ActedIn a1
JOIN ActedIn a2
ON a1.actorname = a2.actorname
WHERE a1.moviename = 'Apocalypse now' AND a2.moviename = 'The god father';

 * sqlite:///dsc100.db
Done.


actorname


Retrieve the names of movies that were released in the 70s and the corresponding actors who acted in them:


In [ ]:
%%sql

SELECT Movie.name, ActedIn.actorname
FROM Movie
JOIN ActedIn ON Movie.name = ActedIn.moviename
WHERE Movie.year BETWEEN 1970 AND 1979;

   sqlite:///dsc100.db
 * sqlite:///test.db
Done.


name,actorname
Apocalypse Now,Marlon Brando
The Godfather,Al Pacino
The Godfather,Marlon Brando


The BETWEEN operator is a logical operator that tests whether a value is in range of values. If the value is in the specified range, the BETWEEN operator returns true. The BETWEEN operator can be used in the WHERE clause of the SELECT, DELETE, UPDATE, and REPLACE statements.

Return all movie genres that Marlon Brando has acted in

In [ ]:
%%sql
SELECT DISTINCT genre
FROM   Movie Join ActedIN
ON Movie.name= ActedIN.moviename
WHERE ActedIN.actorname='Marlon Brando';

   sqlite:///dsc100.db
 * sqlite:///test.db
Done.


genre
War
Crime


In [ ]:
%%sql
-- the following returns the entire information about movies
SELECT *
FROM   Movie, ActedIN
WHERE  Movie.name= ActedIN.moviename AND ActedIN.actorname='Marlon Brando';

 * sqlite:///dsc100.db
Done.


name,year,genre,actorname,moviename
Apocalypse Now,1979,War,Marlon Brando,Apocalypse Now
The Godfather,1972,Crime,Marlon Brando,The Godfather


In [ ]:
%%sql
-- the following is wrong

SELECT *
FROM   Movie, ActedIN
WHERE  ActedIN.actorname='Marlon Brando';

   sqlite:///dsc100.db
 * sqlite:///test.db
Done.


name,year,genre,actorname,moviename
Apocalypse Now,1979,War,Marlon Brando,Apocalypse Now
The Godfather,1972,Crime,Marlon Brando,Apocalypse Now
Planet Earth II,2016,Nature Documentary,Marlon Brando,Apocalypse Now
House of Gucci,2021,Crime,Marlon Brando,Apocalypse Now
Apocalypse Now,1979,War,Marlon Brando,The Godfather
The Godfather,1972,Crime,Marlon Brando,The Godfather
Planet Earth II,2016,Nature Documentary,Marlon Brando,The Godfather
House of Gucci,2021,Crime,Marlon Brando,The Godfather


In [ ]:
%%sql
SELECT * FROM ActedIn;

 * sqlite:///dsc100.db
Done.


actorname,moviename
Marlon Brando,Apocalypse Now
Al Pacino,The Godfather
Marlon Brando,The Godfather
Robert De Niro,House of Gucci


In [ ]:
%%sql
SELECT * FROM Movie;

 * sqlite:///dsc100.db
Done.


name,year,genre
Apocalypse Now,1979,War
The Godfather,1972,Crime
Planet Earth II,2016,Nature Documentary
House of Gucci,2021,Crime


In [ ]:
%%sql
SELECT *
FROM ActedIn

 * sqlite:///dsc100.db
Done.


actorname,moviename
Marlon Brando,Apocalypse Now
Al Pacino,The Godfather
Marlon Brando,The Godfather
Robert De Niro,House of Gucci


Retrieve the names of actors who acted in both 'Apocalypse now' and 'The god father':

In [ ]:
%%sql
SELECT *
FROM ActedIn AS a1
JOIN ActedIn AS a2;

 * sqlite:///dsc100.db
Done.


actorname,moviename,actorname_1,moviename_1
Marlon Brando,Apocalypse Now,Marlon Brando,Apocalypse Now
Marlon Brando,Apocalypse Now,Al Pacino,The Godfather
Marlon Brando,Apocalypse Now,Marlon Brando,The Godfather
Marlon Brando,Apocalypse Now,Robert De Niro,House of Gucci
Al Pacino,The Godfather,Marlon Brando,Apocalypse Now
Al Pacino,The Godfather,Al Pacino,The Godfather
Al Pacino,The Godfather,Marlon Brando,The Godfather
Al Pacino,The Godfather,Robert De Niro,House of Gucci
Marlon Brando,The Godfather,Marlon Brando,Apocalypse Now
Marlon Brando,The Godfather,Al Pacino,The Godfather


In [ ]:
%%sql
SELECT *
FROM ActedIn AS a1
JOIN ActedIn AS a2
ON a1.actorname = a2.actorname;

 * sqlite:///dsc100.db
Done.


actorname,moviename,actorname_1,moviename_1
Marlon Brando,Apocalypse Now,Marlon Brando,Apocalypse Now
Marlon Brando,Apocalypse Now,Marlon Brando,The Godfather
Al Pacino,The Godfather,Al Pacino,The Godfather
Marlon Brando,The Godfather,Marlon Brando,Apocalypse Now
Marlon Brando,The Godfather,Marlon Brando,The Godfather
Robert De Niro,House of Gucci,Robert De Niro,House of Gucci


In [ ]:
%%sql
SELECT *
FROM ActedIn AS a1
JOIN ActedIn AS a2
ON a1.actorname = a2.actorname
WHERE a1.moviename='Apocalypse Now';

 * sqlite:///dsc100.db
Done.


actorname,moviename,actorname_1,moviename_1
Marlon Brando,Apocalypse Now,Marlon Brando,Apocalypse Now
Marlon Brando,Apocalypse Now,Marlon Brando,The Godfather


In [ ]:
%%sql
SELECT a1.actorname
FROM ActedIn AS a1
JOIN ActedIn AS a2
ON a1.actorname = a2.actorname
WHERE a1.moviename='Apocalypse Now' AND a2.moviename='The Godfather';

 * sqlite:///dsc100.db
Done.


actorname
Marlon Brando


In [ ]:
%%sql
SELECT *
FROM ActedIn AS a1
JOIN ActedIn AS a2
ON a1.actorname = a2.actorname
WHERE a1.moviename='Apocalypse Now' AND a2.moviename='The Godfather';

 * sqlite:///dsc100.db
Done.


actorname,moviename,actorname_1,moviename_1
Marlon Brando,Apocalypse Now,Marlon Brando,The Godfather


In [ ]:
%%sql
SELECT actorname
FROM ActedIn AS a1
JOIN ActedIn AS a2
ON a1.actorname = a2.actorname
WHERE a1.moviename='Apocalypse Now' AND a2.moviename='The Godfather';

 * sqlite:///dsc100.db
(sqlite3.OperationalError) ambiguous column name: actorname
[SQL: SELECT actorname
FROM ActedIn AS a1
JOIN ActedIn AS a2
ON a1.actorname = a2.actorname
WHERE a1.moviename='Apocalypse Now' AND a2.moviename='The Godfather';]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [ ]:
%%sql
SELECT a2.actorname
FROM ActedIn AS a1
JOIN ActedIn AS a2
ON a1.actorname = a2.actorname
WHERE a1.moviename='Apocalypse Now' AND a2.moviename='The Godfather';

 * sqlite:///dsc100.db
Done.


actorname
Marlon Brando


Retrieve all actor names that have acted in at least one movie in both ‘Crime’ and ‘War’ categories.

In [ ]:
%%sql
-- is this correct?
SELECT DISTINCT actorname
FROM   Movie, ActedIN
WHERE  Movie.name= ActedIN.moviename AND (Movie.genre='Crime' AND Movie.genre= 'War');


 * sqlite:///dsc100.db
Done.


actorname


In [ ]:
%%sql
-- how about this one?
SELECT DISTINCT actorname
FROM   Movie, ActedIN
WHERE  Movie.name= ActedIN.moviename AND (Movie.genre='Crime' OR Movie.genre= 'War');



 * sqlite:///dsc100.db
Done.


actorname
Al Pacino
Marlon Brando
Robert De Niro


In [ ]:
%%sql
-- using  self-join: attempt 1

SELECT DISTINCT z.actorname
FROM   Movie x, Movie y, ActedIN z
WHERE  x.name= z.moviename
AND    y.name= z.moviename  AND
       x.genre='Crime' AND y.genre='War';

 * sqlite:///dsc100.db
Done.


actorname


In [ ]:
%%sql
-- using  self-join: attempt 2

SELECT DISTINCT z.actorname
FROM   Movie x, Movie y, ActedIN z, ActedIN w
WHERE  x.name= z.moviename
AND    y.name= w.moviename
AND    z.actorname=w.actorname AND
       x.genre='Crime' AND y.genre='War';

 * sqlite:///dsc100.db
Done.


actorname
Marlon Brando


In [ ]:
%%sql
SELECT *
FROM   Movie x, Movie y, ActedIN z, ActedIN w
WHERE  x.name= z.moviename
AND    y.name= w.moviename
AND    z.actorname=w.actorname AND
       x.genre='Crime' AND y.genre='War';


 * sqlite:///dsc100.db
Done.


name,year,genre,name_1,year_1,genre_1,actorname,moviename,actorname_1,moviename_1
The Godfather,1972,Crime,Apocalypse Now,1979,War,Marlon Brando,The Godfather,Marlon Brando,Apocalypse Now


Retrieve ALL movie names, their genres and their actors that produced after 1975.

In [ ]:
%%sql

-- what's wrong with the following?

SELECT DISTINCT  name, genre, actorname
FROM   Movie Join ActedIN
WHERE  Movie.name= ActedIN.moviename AND Movie.year> 1975;

   sqlite:///dsc100.db
 * sqlite:///test.db
Done.


name,genre,actorname
Apocalypse Now,War,Marlon Brando
House of Gucci,Crime,Robert De Niro


In [ ]:
%%sql
-- we need outer-join
SELECT   * FROM   Movie LEFT OUTER JOIN ActedIN
on  Movie.name= ActedIN.moviename AND Movie.year> 1975;

   sqlite:///dsc100.db
 * sqlite:///test.db
Done.


name,year,genre,actorname,moviename
Apocalypse Now,1979,War,Marlon Brando,Apocalypse Now
The Godfather,1972,Crime,None,None
Planet Earth II,2016,Nature Documentary,None,None
House of Gucci,2021,Crime,Robert De Niro,House of Gucci


In [ ]:
%%sql
SELECT   * FROM   Movie
LEFT OUTER JOIN ActedIN
WHERE  Movie.name= ActedIN.moviename AND Movie.year> 1975;

   sqlite:///dsc100.db
 * sqlite:///test.db
Done.


name,year,genre,actorname,moviename
Apocalypse Now,1979,War,Marlon Brando,Apocalypse Now
House of Gucci,2021,Crime,Robert De Niro,House of Gucci


In [ ]:
%%sql
SELECT   * FROM   Movie JOIN ActedIN on  Movie.name= ActedIN.moviename WHERE Movie.year> 1975;


 * sqlite:///dsc100.db
Done.


name,year,genre,actorname,moviename
Apocalypse Now,1979,War,Marlon Brando,Apocalypse Now
House of Gucci,2021,Crime,Robert De Niro,House of Gucci


In [ ]:
%%sql
SELECT   * FROM   Movie JOIN ActedIN WHERE Movie.name= ActedIN.moviename AND Movie.year> 1975;


 * sqlite:///dsc100.db
Done.


name,year,genre,actorname,moviename
Apocalypse Now,1979,War,Marlon Brando,Apocalypse Now
House of Gucci,2021,Crime,Robert De Niro,House of Gucci


Here is another example that illustrates outer joins:

Let's start with two tables: Employee(id, name) and Sales(employeeID, productID)
The tables have the following content
(1,'Joe')         (1, 344)
(2,'Jack')        (1, 355)
(3,'Gill')        (2, 544)
If we run a simple join, Gill will not appear in the result because she did not make any sells. If we run a left outer-join, Gill will be returned with a null sale. We can similarly do right outer joins and full outer joins (but not in sqlite)

In [ ]:
%%sql
DROP TABLE Employee;
DROP TABLE Sales;
CREATE TABLE Employee (
    id INT,
    name VARCHAR(10)
);

CREATE TABLE Sales (
    employeeID INT,
    productID INT
);

INSERT INTO Employee VALUES (1,'Joe');
INSERT INTO Employee VALUES (2,'Jack');
INSERT INTO Employee VALUES (3,'Gill');

INSERT INTO Sales VALUES (1,344);
INSERT INTO Sales VALUES (1,355);
INSERT INTO Sales VALUES (2,544);


 * sqlite:///dsc100.db
Done.
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [ ]:
%%sql
SELECT *
FROM Employee E
JOIN Sales S
ON E.id = S.employeeID;


 * sqlite:///dsc100.db
Done.


id,name,employeeID,productID
1,Joe,1,344
1,Joe,1,355
2,Jack,2,544


In [ ]:
%%sql
SELECT *
FROM Employee E
INNER JOIN Sales S
ON E.id = S.employeeID;


 * sqlite:///dsc100.db
Done.


id,name,employeeID,productID
1,Joe,1,344
1,Joe,1,355
2,Jack,2,544


In [ ]:
%%sql
SELECT *
FROM Employee E
LEFT OUTER JOIN Sales S
ON E.id = S.employeeID;


 * sqlite:///dsc100.db
Done.


id,name,employeeID,productID
1,Joe,1,344
1,Joe,1,355
2,Jack,2,544
3,Gill,None,None


Retrieve the product IDs of all products sold by 'Joe':

In [ ]:
%%sql
SELECT Sales.productID
FROM Sales
JOIN Employee ON Sales.employeeID = Employee.id
WHERE Employee.Name = 'Joe';


 * sqlite:///dsc100.db
Done.


productID
344
355


In [ ]:
%%sql
SELECT E.name FROM Employee E
INNER JOIN Sales S ON E.id = S.employeeID
WHERE S.productID = 544;

 * sqlite:///dsc100.db
Done.


name
Jack


In [ ]:
%%sql
SELECT DISTINCT E.name
FROM Employee E
LEFT JOIN Sales S
ON E.id = S.employeeID
WHERE S.productID != 544;

 * sqlite:///dsc100.db
Done.


name
Joe


In [ ]:
%%sql
SELECT DISTINCT E.name
FROM Employee E
LEFT OUTER JOIN Sales S
ON E.id = S.employeeID
WHERE S.productID != 544;

 * sqlite:///dsc100.db
Done.


name
Joe


Retrieve the names of all employees who did not sell a specific product (product ID 544 in this example):


In [ ]:
%%sql
SELECT DISTINCT E.name
FROM Employee E
LEFT OUTER JOIN Sales S
ON E.id = S.employeeID
WHERE S.productID != 544 OR S.productID IS NULL;

 * sqlite:///dsc100.db
Done.


name
Joe
Gill


 Retrieve the names of all employees who sold a specific product (product ID 544 in this example):